In [4]:
import stim
import random
import time
import os
import math
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator


import stim

folder_path = "/Users/garn195/Project Repositories/NWQ-Sim/stabilizer/stim_to_qasm_files"
# Ensure the folder exists
if not os.path.isdir(folder_path):
    print(f"The folder '{folder_path}' does not exist.")

# List all files in the folder
qasm_files = [f for f in os.listdir(folder_path) if f.endswith('.qasm')]

if not qasm_files:
    print("No .qasm files found in the folder.")

for qasm_file in qasm_files:
    full_path = os.path.join(folder_path, qasm_file)

    qc = QuantumCircuit.from_qasm_file(full_path)

    sim = AerSimulator(method="stabilizer")
    n_qubits = qc.num_qubits

    if n_qubits < 5000:
        start = time.perf_counter()
        result = sim.run(qc).result()
        end = time.perf_counter()
        elapsed = end - start

        output_dir = "/Users/garn195/Project Repositories/NWQ-Sim/stabilizer/surface_operation_bench"

        os.makedirs(output_dir, exist_ok=True)
        filename = os.path.join(output_dir, f"qiskit_{n_qubits}.txt")
        with open(filename, "w") as f:
            f.write("qiskit\n")
            f.write(f"{elapsed}\n")
            f.write(f"{n_qubits}\n")
    
    print("qiskit done\n")

    circuit = stim.Circuit()

    # Iterate over the Qiskit circuit data in a more optimized manner
    for inst, qargs, _ in qc.data:
            # Find the qubit indices
        qargs_indices = [qc.qubits.index(qubit) for qubit in qargs]

        # Map Qiskit gates to Stim gates
        if inst.name == 'h':
            circuit.append("H", qargs_indices)
        elif inst.name == 'cx':
            if len(qargs) % 2 == 0:  # Ensure we have an even number of qubits for CX gates
                for i in range(0, len(qargs_indices), 2):
                    circuit.append("CX", [qargs_indices[i], qargs_indices[i+1]])
            else:
                raise ValueError("CX gate requires an even number of qubits, but got an odd number.")
        elif inst.name == 's':
            circuit.append("S", qargs_indices)
        elif inst.name == 'reset':
            circuit.append("R", qargs_indices)
        elif inst.name == 'measure':
            circuit.append("M", qargs_indices)
        elif inst.name in {'barrier', 'id'}:
            # Skip no-op or unsupported gates that are safe to ignore
            continue
        else:
            raise NotImplementedError(f"Gate '{inst.name}' is not supported by Stim")

    
    simulator = stim.TableauSimulator()

    if n_qubits < 15000:
        start = time.perf_counter()
        simulator.do_circuit(circuit)
        end = time.perf_counter()
        elapsed = end - start

        os.makedirs(output_dir, exist_ok=True)
        filename = os.path.join(output_dir, f"stim_{n_qubits}.txt")
        with open(filename, "w") as f:
            f.write("stim\n")
            f.write(f"{elapsed}\n")
            f.write(f"{n_qubits}\n")
    
    print("stim done\n")


qiskit done



/var/folders/b5/cbnlm0vn1j54xy3c75jt4gwc0000gn/T/ipykernel_66151/1781300738.py:51: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for inst, qargs, _ in qc.data:


KeyboardInterrupt: 